## Run this file on a cloud service if you dont have a compatible GPU/CPU, Used PaperSpace A4000 for this one

In [ ]:
# !pip install transformers
# !pip install langchain-huggingface
# !pip install torch

## PaperSpace

In [ ]:
# !pip install transformers==4.52.4
# !pip install langchain-huggingface
# !pip install torch
# !pip install accelerate
# !pip install gcsa

In [ ]:
import torch

### run this line if no kernel found to launch

# torch.backends.cuda.enable_mem_efficient_sdp(False)
# torch.backends.cuda.enable_flash_sdp(False)

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from transformers import AutoModelForCausalLM, AutoTokenizer
from gcsa.google_calendar import GoogleCalendar
from gcsa.event import Event
from datetime import datetime, timedelta
import re
import json

### Load Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def chat_with_qwen(user_input, system_prompt="You are a helpful AI assistant."):
    messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_input}
    ]

    text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=128
    )
    generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [ ]:
system_prompt = """
You are an AI assistant that extracts meeting information from natural language.

Respond ONLY with one JSON object that looks like:
{{
  "intent": "schedule_event",
  "title": "Meeting with Alice",
  "day": "monday",
  "time": "2pm",
  "participants": "Alice"
}}

DO NOT REPEAT or explain. Just return the JSON.
"""


prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("{user_input}")
])

chain = prompt | llm | StrOutputParser()

response = chain.invoke({"user_input": "Can you schedule a meeting with Jason next Tuesday at 3pm"})


# Main Code


In [ ]:
qwen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=64,
    do_sample=True,
    torch_dtype="auto",
    device_map="auto",
    return_full_text=False,
    temperature= 0.7,
    top_p= 0.8,
    top_k= 20,
    repetition_penalty= 1.05
    )

# temperature to control randomness. lower = more deterministic
# top_p limits choices to top X% of likely tokens Lower = more deterministic
# top_k sample from the top-k most likely tokens. Lower = more deterministic
# do_sample enables sampling

llm = HuggingFacePipeline(pipeline=qwen_pipe)

## Conversational Chatbot Functions

In [ ]:
def clean_generated_response(text):
    # Remove any "Human:" and after it

    return re.split(r"Human:", text)[0].strip()

In [ ]:
def schedule_event(data):
    print("Scheduled event:")
    print(json.dumps(data, indent=2))

In [ ]:
def extract_intent(response: str):
    # Look for JSON-like object
    match = re.search(r'\{.*?\}', response, re.DOTALL)
    if match:
        try:
            data = json.loads(match.group(0))
            if all(k in data for k in ["title", "date", "time", "description"]):
                return {"action": "schedule", "data": data}
        except json.JSONDecodeError:
            pass
    return {"action": "chat", "data": response}


intent_parser = RunnableLambda(extract_intent)


In [ ]:
chain = llm | intent_parser

In [ ]:
## For Calendar authentication

calendar = GoogleCalendar(credentials_path='credentials.json')

In [ ]:
from gcsa.event import Eventfrom gcsa.event import Event
from datetime import datetime

### Event class parameter
# classgcsa.event.Event(summary,
#                       start,
#                       end=None,
#                       *,
#                       timezone=get_localzone_name(),
#                       event_id=None,
#                       description=None,
#                       location=None,
#                       recurrence=None,
#                       color_id=None,
#                       visibility=Visibility.DEFAULT,
#                       attendees=None,
#                       attachments=None,
#                       conference_solution=None,
#                       reminders=None,
#                       default_reminders=False,
#                       minutes_before_popup_reminder=None,
#                       minutes_before_email_reminder=None,
#                       guests_can_invite_others=True,
#                       guests_can_modify=False,
#                       guests_can_see_other_guests=True,
#                       transparency=None,
#                       _creator=None,
#                       _organizer=None,
#                       _created=None,
#                       _updated=None,
#                       _recurring_event_id=None,
#                       **other)

def handle_schedule_confirmation(schedule_data, calendar, raw_response=None):
    title = schedule_data['title']
    date = schedule_data['date']
    time = schedule_data['time']
    duration = schedule_data.get('duration', 1) # default 1 hour
    description = schedule_data['description']

    # Convert to datetime object
    start_dt = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
    end_dt = start_dt + timedelta(hours= duration)

    event = Event(
        summary=title,
        start=start_dt,
        end = end_dt,
        description=description
    )

    calendar.add_event(event)
    if raw_response:
        print(f"AI: {raw_response}")
    else:
        print("AI: Event added to Google Calendar! Is there anything else I can help you with?")



from datetime import datetime

### Event class parameter
# classgcsa.event.Event(summary,
#                       start,
#                       end=None,
#                       *,
#                       timezone=get_localzone_name(),
#                       event_id=None,
#                       description=None,
#                       location=None,
#                       recurrence=None,
#                       color_id=None,
#                       visibility=Visibility.DEFAULT,
#                       attendees=None,
#                       attachments=None,
#                       conference_solution=None,
#                       reminders=None,
#                       default_reminders=False,
#                       minutes_before_popup_reminder=None,
#                       minutes_before_email_reminder=None,
#                       guests_can_invite_others=True,
#                       guests_can_modify=False,
#                       guests_can_see_other_guests=True,
#                       transparency=None,
#                       _creator=None,
#                       _organizer=None,
#                       _created=None,
#                       _updated=None,
#                       _recurring_event_id=None,
#                       **other)

def handle_schedule_confirmation(schedule_data, calendar, raw_response=None):
    title = schedule_data['title']
    date = schedule_data['date']
    time = schedule_data['time']
    duration = schedule_data.get('duration', 1) # default 1 hour
    description = schedule_data['description']

    # Convert to datetime object
    start_dt = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
    end_dt = start_dt + timedelta(hours= duration)

    event = Event(
        summary=title,
        start=start_dt,
        end = end_dt,
        description=description
    )

    calendar.add_event(event)
    if raw_response:
        print(f"AI: {raw_response}")
    else:
        print("AI: Event added to Google Calendar! Is there anything else I can help you with?")




## Conversational Chatbot

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

pending_schedule = None

chat_history = []

# initial system message (general context) with a few shot prompt to guide the model
chat_history = [SystemMessage(content=(
                    "You are a helpful assistant.\n\n"
                    "Do NOT include role names like 'Human:', 'AI:', or 'Assistant:' in your responses.\n"
                    "If the user asks to schedule something (like a meeting, reminder, or event), "
                    "respond ONLY with a single valid JSON object on its own line, with this exact format:\n"
                    "{\n"
                    '  "title": "Meeting with John",\n'
                    '  "date": "2025-07-02",\n'
                    '  "time": "15:00",\n'
                    '  "duration": 1,\n'
                    '  "description": "Discuss project updates"\n'
                    "}\n\n"
                    "ONLY and ALWAYS ask confirmation after the JSON.\n"
                    "DO NOT explain.\n"
                    "Just print the JSON block and wait for confirmation."
                    )

                ),
                HumanMessage(
                    content=(
                        "Hi what are you?"
                    )
                ),
                AIMessage(
                    content=(
                        "I am your helpful AI assistant. I can help you schedule things and please "
                        "give me specific details about title, date, time and description"
                    )
                ),
                HumanMessage(
                    content=(
                        "Give me one book about Finance and Investing for beginner"
                    )
                ),
                AIMessage(
                    content=(
                        "'The Intelligent Investor' by Benjamin Graham is a good book. Would "
                        "you like any more recommendations or any other specific tasks you "
                        "like me to help with?"
                    )
                ),
                HumanMessage(
                    content=(
                        "Schedule a meeting with Jason on Sunday, 2 October 2025, at 3PM, about "
                        "Brawl stars session"
                    )
                ),
                AIMessage(
                    content=(
                          "{\n"
                        '  "title": "Meeting with Jason",\n'
                        '  "date": "2025-09-02",\n'
                        '  "time": "15:00",\n'
                        '  "duration": 1,\n'
                        '  "description": "Brawl stars session"\n'
                        "}\n\n"
                    )
                ),
            ]

while True:
    query = input("Human: ")
    if query.lower().strip() in ["exit","bye","goodbye"]:
        print("System: Goodbye")
        break

    chat_history.append(HumanMessage(content=query))

    if pending_schedule and query.lower().strip() in ["yes", "confirm", "okay"]:
        print("disini 1")
        schedule_event(pending_schedule)

        handle_schedule_confirmation(pending_schedule, calendar)
        continue
    elif pending_schedule and query.lower().strip() in ["no", "cancel"]:
        print("disini 2")
        print("AI: Okay, canceled.")
        pending_schedule = None
        continue

    uncleaned_result = chain.invoke(chat_history)

    # model likes to add 'Human: ...' by itsekf
    if isinstance(uncleaned_result["data"], str):
        uncleaned_result["data"] = clean_generated_response(uncleaned_result["data"])

    result = uncleaned_result

    if result["action"] == "schedule":
        pending_schedule = result["data"]
        print("disini 3")
        print("AI: I found a scheduling request. Please confirm:")
        print(json.dumps(pending_schedule, indent=2))
        chat_history.append(AIMessage(content=json.dumps(pending_schedule)))

    else:
        print("disini 4")
        print(f"{result['data']}")
        chat_history.append(AIMessage(content=result["data"]))

hi what are you?

give me one book about cooking

can you schedule a meeting with bob on saturday 4 october 2025, about brawl stars ranked session